In [3]:
import requests
from time import sleep
from ecc import PrivateKey
from helper import decode_bech32, decode_base58, hash256, little_endian_to_int
from network import SimpleNode
from script import p2wpkh_script, p2sh_script
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
my_address = private_key.point.bech32_address(testnet=True)
target_amount = 5000000
print(my_address)
response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')
student_addresses = '''2N4MoSx2SoExv53GJFEdPuMqL8djPQPH2er'''
fee = 1000
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    if target_address.startswith('2'):
        h160 = decode_base58(target_address)
        script_pubkey = p2sh_script(h160)
    elif target_address.startswith('t'):
        # decode the hash160 from the target address
        _, _, h160 = decode_bech32(target_address)
        # convert hash160 to p2pkh script
        script_pubkey = p2wpkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=script_pubkey,
    ))
change_satoshis = total - fee - output_amount
script_pubkey = p2wpkh_script(private_key.point.hash160())
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True, segwit=True)
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, private_key)
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))
print(tx_obj.serialize().hex())
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
node.handshake()
node.send(tx_obj)
if node.is_tx_accepted(tx_obj):
    print('success! {}'.format(tx_obj.id()))

031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e


RuntimeError: 

In [7]:
import requests
from time import sleep
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from network import SimpleNode
from script import Script, p2wsh_script
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
my_sec = private_key.point.sec()
my_address = private_key.point.bech32_address(testnet=True)
target_amount = 5000000
print(my_address)
response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')
student_secs = '''02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d'''
fee = 1000
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
witness_scripts = []
secs = student_secs.split()
for sec_hex in secs:
    sec = bytes.fromhex(sec_hex)
    witness_script = Script([0x52, sec, my_sec, 0x52, 0xae])
    witness_scripts.append(witness_script)
    _, _, s256 = decode_bech32(address)
    script_pubkey = p2wsh_script(s256)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=script_pubkey,
    ))
change_amount = total - fee - output_amount
script_pubkey = p2wpkh_script(private_key.point.hash160())
tx_outs.append(TxOut(amount=change_amount, script_pubkey=script_pubkey))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True, segwit=True)
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, private_key)
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))
print(tx_obj.serialize().hex())
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
node.handshake()
node.send(tx_obj)
if node.is_tx_accepted(tx_obj):
    print('success! {}'.format(tx_obj.id()))
prev_tx = tx_obj.hash()
# now create transactions to send everything back
for prev_index, sec_hex, witness_script in zip(range(len(secs)), secs, witness_scripts):
    tx_in = TxIn(prev_tx, prev_index)
    tx_out = TxOut(4999500, script_pubkey)
    t = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
    sig = t.get_sig_p2wsh_multisig(0, private_key, witness_script=witness_script)
    address = witness_script.p2wsh_address(testnet=True)
    print('{},{},{},{}'.format(sec_hex, address, t.serialize().hex(), sig.hex()))

tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au
0100000000010195e1e2e438a3125e432b85a629445979096c881c15a3b0e15ef2fcbd39e1448a0100000000ffffffff02404b4c0000000000220020b150ad130d9d7d3572e2fbe7397ba436c5f667976091e62db60e3f186ec0e31080e6d801000000001600146e13971913b9aa89659a9f53d327baa8826f2d75024830450221008ac5e6d37284b1963b5295546c44d244d04366e222a21c87b821c2112296b89c02204a2a932abcc8f878e2d399c418acd47655d9ba3944387580cdd988693bd8cba50121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000
success! 2c7482b437e26f07422e0b10f6793824172dc2c24cb524f79a05faa54a58ad74
02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d	tb1qk9g26ycdn47n2uhzl0nnj7ayxmzlveuhvzg7vtdkpcl3smkquvgqlp5t8w	0100000000010174ad584aa5fa059af724b54cc2c22d17243879f6100b2e42076fe237b482742c0000000000ffffffff014c494c00000000001600146e13971913b9aa89659a9f53d327baa8826f2d750000000000	30440220720893922a74aedf13c5c8cf61f2c71512db5339a093aae3293f86877fb351c402206a87ea8b1cfea8cd9356b10d91da1080f7f

In [6]:
import requests
from time import sleep
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from network import SimpleNode
from script import Script, p2wsh_script, p2wpkh_script
from tx import Tx, TxIn, TxOut
passphrase1 = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase1))
pk1 = PrivateKey(secret)
passphrase2 = b'jimmy@programmingblockchain.com secret passphrase 2'
secret = little_endian_to_int(hash256(passphrase2))
pk2 = PrivateKey(secret)
sec1 = pk1.point.sec()
sec2 = pk2.point.sec()
my_address = pk1.point.bech32_address(testnet=True)
target_amount = 5000000
print(my_address)
response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')
student_secs = '''02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d'''
fee = 1000
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
witness_scripts = []
secs = student_secs.split()
for sec_hex in secs:
    sec0 = bytes.fromhex(sec_hex)
    witness_script = Script([0x52, sec0, sec1, sec2, 0x53, 0xae])
    witness_scripts.append(witness_script)
    redeem_script = witness_script.p2wsh_script_pubkey()
    script_pubkey = redeem_script.p2sh_script_pubkey()
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=script_pubkey,
    ))
change_amount = total - fee - output_amount
script_pubkey = p2wpkh_script(pk1.point.hash160())
tx_outs.append(TxOut(amount=change_amount, script_pubkey=script_pubkey))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True, segwit=True)
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, pk1)
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))
print(tx_obj.serialize().hex())
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
node.handshake()
node.send(tx_obj)
if node.is_tx_accepted(tx_obj):
    print('success! {}'.format(tx_obj.id()))
prev_tx = tx_obj.hash()
# now create transactions to send everything back
for prev_index, sec_hex, witness_script in zip(range(len(secs)), secs, witness_scripts):
    tx_in = TxIn(prev_tx, prev_index)
    tx_out = TxOut(4999500, script_pubkey)
    t = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
    sig = t.get_sig_p2wsh_multisig(0, pk2, witness_script)
    address = witness_script.p2sh_p2wsh_address(testnet=True)
    print('{},{},{},{}'.format(sec_hex, address, t.serialize().hex(), witness_script.serialize().hex(), sig.hex()))

tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au
0100000000010274ad584aa5fa059af724b54cc2c22d17243879f6100b2e42076fe237b482742c0100000000ffffffff97318efd64c7cfc0efd64b2c1ce0f0b30a0044526b981916dcc39b09b81edbad0000000000ffffffff02404b4c000000000017a91496504b973dea4b2e956bb4d6f255afd0768bdb4a87a4d79b01000000001600146e13971913b9aa89659a9f53d327baa8826f2d75024730440220104b8a4b909e44b30a6e1ba49cba872788ff071f6eb81900475a061a0016efd402203bca37c1528b10734571be4e2d287b86a1852827b44ae6eb8978d59cbfa15ef90121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e02483045022100c9c84015f3a8f2c23afac5f368b02b9e3ec2d95b6142fc86a73ef7df70b3a08b02204c9dc4e18b1a20a7ce571890485e329c88b8f337ffac6bbc7137f0f90119256c0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000
success! 3d9425b02f018d700c2b965e44daa333657de43b771760c8bc00a90caffc730b
02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d,2N6x1Y58uwZ7oYAZSeYveaYHGKHvotUUfyZ,010000000001010b73fcaf0ca900bcc8601

In [11]:
from io import BytesIO
from ecc import PrivateKey
from tx import Tx
passphrase = b'jimmy@programmingblockchain.com secret passphrase 2'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
hex_tx = '01000000000101ec13653fae5706168e92d9a3e4d98044d4af001a3081a70e57865caf94e0b7ee0000000000ffffffff014c400f00000000001600146e13971913b9aa89659a9f53d327baa8826f2d750000000000'
stream = BytesIO(bytes.fromhex(hex_tx))
tx_obj = Tx.parse(stream, testnet=True)
hex_witness_script = '69522102c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d21031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e2102618b836fc32578538bb8440f5e89d916844dd828981a9bc33f9a736638b538d253ae'
stream = BytesIO(bytes.fromhex(hex_witness_script))
witness_script = Script.parse(stream)
sig = tx_obj.get_sig_p2wsh_multisig(0, private_key, witness_script)
print(sig.hex())

3045022100e9fa8587958b540ac71f128629e3019f07b14b0e5eb248afb3d4f92ddfee9f6002201bda6dc9297471419a00d86899f753d6c015e42d51df9aa1d8b5dc53f595d9e401
